# ● 初期設定（最初の一回のみ実行）

#### ***Python3.10.6 環境の準備　（このコマンドのみターミナルで実行）***

In [ ]:
conda create -y --name sd_env python=3.10.6
conda activate sd_env
conda install -y ipykernel

#### ***Stable Diffusion WebUI　の導入（カーネルを『sd_env』に切り替えて実行）***

In [ ]:
%cd /home/studio-lab-user/
!python -V

# 必要なライブラリのインストール
%conda install -y -c conda-forge libglib==2.78.0 aria2==1.36.0
!pip install triton==2.0.0 httpx==0.24.1 matplotlib==3.8.1 ipython==8.17.2 -U
!pip install torchvision==0.15.2 -U
!pip install xformers==0.0.20 -U

import os
import glob
from IPython import get_ipython

# グラフ描画ライブラリをインラインで表示する設定
get_ipython().run_line_magic('matplotlib', 'inline')

# Stable Diffusion WebUI のリポジトリが無ければクローン
if not os.path.isdir("/home/studio-lab-user/ui"):
    %cd /home/studio-lab-user
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /home/studio-lab-user/ui
    # Stable Diffusion WebUI の v1.6.0 を使用する。
    %cd /home/studio-lab-user/ui
    !git checkout master
    !git checkout 5ef669de080814067961f28357256e8fe27544f4

# ● Stable Diffusion WebUI を起動

In [ ]:
# Modelがなければ sd_xl_base_1.0.safetensors をダウンロード
if not glob.glob("/home/studio-lab-user/ui/models/Stable-diffusion/*.safetensors"):
    os.system("aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -d /home/studio-lab-user/ui/models/Stable-diffusion -o sd_xl_base_1.0.safetensors")
    
# 設定を変更
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /home/studio-lab-user/ui/modules/shared_options.py

if os.path.exists("/home/studio-lab-user/ui/ui-config.json"):
    !sed -i -e 's/value": 512/value": 1024/g' /home/studio-lab-user/ui/ui-config.json
    !sed -i -e 's/Grid margins (px)\/value": 0/Grid margins (px)\/value": 5/g' /home/studio-lab-user/ui/ui-config.json

if os.path.exists("/home/studio-lab-user/ui/config.json"):
    !sed -i -e 's/"\[date\]"/"\[date\]-\[model_name\]"/g' /home/studio-lab-user/ui/config.json

# NGROK TOKEN を定義
ngrok_token="YOUR NGROK TOKEN"
ngrok_token="2SOs8DsoZvkiDNBFjisetiIZJqu_7XBuj79rA4yeX3Bxas3Dc"

# Stable Diffusion WebUI を起動
%cd /home/studio-lab-user/ui
ARGS = "\"--listen --xformers --enable-insecure-extension-access --no-half-vae --ngrok " + ngrok_token + "\""
!COMMANDLINE_ARGS=$ARGS python launch.py

# ● Options

#### ***Model, VAE のダウンロード***

In [ ]:
"""
ダウンロードしないものはコメントアウトする。
コメントアウトの方法 → 行頭に『# 』をつける。
"""

# ダウンロード方法を定義
ARIA2="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
# Modelのディレクトリを定義
MODEL_DIR="/home/studio-lab-user/ui/models/Stable-diffusion"
!rm -rf {MODEL_DIR}/* #学習モデル を全て削除（リセット）

# blue_pencil　
!{ARIA2} https://huggingface.co/bluepen5805/blue_pencil-XL/resolve/main/blue_pencil-XL-v0.8.0.safetensors -d {MODEL_DIR} -o blue_pencil-XL-v0.8.0.safetensors

# sdvn7Nijistylexl_v1
# !{ARIA2} https://civitai.com/api/download/models/155870 -d {MODEL_DIR} -o sdvn7Nijistylexl_v1.safetensors

# CherryPickerXL26　
# !{ARIA2} https://civitai.com/api/download/models/173768 -d {MODEL_DIR} -o CherryPickerXL26.safetensors

# Refiner
# !{ARIA2} https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -d {MODEL_DIR} -o sd_xl_refiner_1.0.safetensors

# SD2.0以前
# !{ARIA2} https://civitai.com/api/download/models/119057 -d {MODEL_DIR} -o MeinaMix.safetensors
# !{ARIA2} https://civitai.com/api/download/models/113479 -d {MODEL_DIR} -o beautifulRealistic_v60.safetensors


# VAE
VAE_DIR="/home/studio-lab-user/ui/models/VAE"
!rm {VAE_DIR}/*
# !{ARIA2} https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors -d {VAE_DIR} -o sdxl_vae.vae.safetensors


# 確認
!echo -= MODEL_DIR を確認 =- && ls -l {MODEL_DIR}
!echo -= VAE_DIR を確認 =- && ls -l {VAE_DIR}

#### ***LoRA, Embedings のダウンロード***

In [ ]:
"""
ダウンロードしないものはコメントアウトする。
コメントアウトの方法 → 行頭に『# 』をつける。
"""

# ダウンロード方法を定義
ARIA2="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
# LoRA のディレクトリを定義
LORA_DIR="/home/studio-lab-user/ui/models/Lora" 
# Embedings のディレクトリを定義
EMBEDDINGS_DIR="/home/studio-lab-user/ui/embeddings"


# LoRA のダウンロード ######################################################################
!rm -rf {LORA_DIR}/*  #LoRA を全て削除（リセット）
!{ARIA2} https://civitai.com/api/download/models/155932 -d {LORA_DIR} -o ClearHand-V2.safetensors
# !{ARIA2} https://civitai.com/api/download/models/62833 -d {LORA_DIR} -o detail-tweaker-xl.safetensors


# Embedings のダウンロード ######################################################################
!rm -rf {EMBEDDINGS_DIR}/*  #Embedings を全て削除（リセット）
!{ARIA2} https://civitai.com/api/download/models/175819 -d {EMBEDDINGS_DIR} -o unaestheticXL_Sky3.1.safetensors
!{ARIA2} https://civitai.com/api/download/models/162146 -d {EMBEDDINGS_DIR} -o unaestheticXL_AYv1.safetensors
!{ARIA2} https://civitai.com/api/download/models/149308 -d {EMBEDDINGS_DIR} -o unaestheticXLv31.safetensors


# 確認
!echo -= LORA_DIR を確認 =- && ls -l {LORA_DIR}
!echo -= EMBEDDINGS_DIR を確認 =- && ls -l {EMBEDDINGS_DIR}

#### ***拡張機能のインストール***

In [ ]:
"""
インストールしないものはコメントアウトする。
コメントアウトの方法 → 行頭に『# 』をつける。
"""

# 拡張機能 のディレクトリを定義
EXTENSIONS_DIR = "/home/studio-lab-user/ui/extensions"
# !rm -rf {EXTENSIONS_DIR}/*  #拡張機能 を全て削除（リセット）

# 拡張機能のインストール
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {EXTENSIONS_DIR}/images-browser
!git clone https://github.com/zanllp/sd-webui-infinite-image-browsing {EXTENSIONS_DIR}/infinite-image-browsing
!git clone https://github.com/Haoming02/sd-webui-prompt-format {EXTENSIONS_DIR}/prompt-format
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete {EXTENSIONS_DIR}/tagcomplete
!git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger {EXTENSIONS_DIR}/wd14-tagger
!git clone https://github.com/Physton/sd-webui-prompt-all-in-one {EXTENSIONS_DIR}/prompt-all-in-one
!git clone https://github.com/Bing-su/adetailer {EXTENSIONS_DIR}/adetailer
!git clone https://github.com/Mikubill/sd-webui-controlnet {EXTENSIONS_DIR}/sd-webui-controlnet
!git clone https://github.com/anonCantCode/sd-webui-send-to-controlnet {EXTENSIONS_DIR}/send-to-controlnet
!git clone https://github.com/jexom/sd-webui-depth-lib {EXTENSIONS_DIR}/depth-lib
!git clone https://github.com/IDEA-Research/DWPose {EXTENSIONS_DIR}/DWPose
!git clone https://github.com/huchenlei/sd-webui-openpose-editor {EXTENSIONS_DIR}/openpose-editor
!git clone https://github.com/ljleb/sd-webui-freeu {EXTENSIONS_DIR}/freeu
!git clone https://github.com/alemelis/sd-webui-ar {EXTENSIONS_DIR}/sd-webui-ar
!git clone https://github.com/yankooliveira/sd-webui-photopea-embed  {EXTENSIONS_DIR}/photopea-embed
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg {EXTENSIONS_DIR}/rembg
# !git clone https://github.com/aka7774/sd_katanuki {EXTENSIONS_DIR}/sd_katanuki
# !git clone https://github.com/AlUlkesh/sd_save_intermediate_images {EXTENSIONS_DIR}/sd_save_intermediate_images

# wd14-tagger のエラー対策（コミット → 99bf7d8132cb757edbcdfaad654f31f32f3500a5）
!sed -i -e 's/from webui import wrap_gradio_gpu_call/from modules.call_queue import wrap_gradio_gpu_call/g' /home/studio-lab-user/ui/extensions/wd14-tagger/tagger/ui.py
!sed -i -e 's/from modules.shared import models_path/import argparse\nimport os\n\nmodules_path = os.path.dirname(os.path.realpath(__file__))\n\nparser_pre = argparse.ArgumentParser(add_help=False)\nparser_pre.add_argument("--data-dir", type=str, default=os.path.dirname(modules_path), help="base path where all user data is stored", )\ncmd_opts_pre = parser_pre.parse_known_args()[0]\ndata_path = cmd_opts_pre.data_dir\n\nmodels_path = os.path.join(data_path, "models")/g' /home/studio-lab-user/ui/extensions/wd14-tagger/preload.py

# sd-webui-ar の拡張機能に SDXL の推奨サイズを書き込み
f = open("/home/studio-lab-user/ui/extensions/sd-webui-ar/resolutions.txt", "w")
text = """SDv2, 512, 512 # SDXL以前のサイズ
SDXL 1:1, 1024, 1024 # Square
SDXL 2:3, 836, 1254 # Landscape
SDXL 3:4, 886, 1182 # Photo
SDXL 9:16, 768, 1344 # Vertical"""
f.write(text)
f.close()

# 確認
!echo -= EXTENSIONS_DIR を確認 =- && ls -l {EXTENSIONS_DIR}

#### ***SDXL版 ControlNet Model, ADetailer Model のダウンロード***

In [ ]:
"""
インストールしないものはコメントアウトする。
コメントアウトの方法 → 行頭に『# 』をつける。
"""

# ダウンロード方法を定義
ARIA2="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
# Hugging Face の ControlNet Model の共通のダウンロードURLを定義
HF_CTN_URL="https://huggingface.co/lllyasviel/sd_control_collection/resolve/main"


# ControlNet Model のダウンロード ---------------------------------------------------
CTN_MODEL_DIR = "/home/studio-lab-user/ui/models/ControlNet"
!rm -rf {CTN_MODEL_DIR}/* #ControlNet Model を全て削除（リセット）

# Canny
# !{ARIA2} {HF_CTN_URL}/sai_xl_canny_256lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_canny_256lora.safetensors #774 MB
# !{ARIA2} {HF_CTN_URL}/sai_xl_canny_128lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_canny_128lora.safetensors #396 MB
# !{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_canny.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_canny.safetensors #46.2 MB

# Depth
# !{ARIA2} {HF_CTN_URL}/sai_xl_depth_256lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_depth_256lora.safetensors #774 MB
# !{ARIA2} {HF_CTN_URL}/sai_xl_depth_128lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_depth_128lora.safetensors #396 MB
!{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_depth.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_depth.safetensors #46.2 MB

# Openpose
# !{ARIA2} {HF_CTN_URL}/thibaud_xl_openpose_256lora.safetensors -d {CTN_MODEL_DIR} -o thibaud_xl_openpose_256lora.safetensors #774 MB
!{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_openpose_anime_v2.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_openpose_anime_v2.safetensors #46.2 MB

# Lineart
!{ARIA2} {HF_CTN_URL}/t2i-adapter_diffusers_xl_lineart.safetensors -d {CTN_MODEL_DIR} -o t2i-adapter_diffusers_xl_lineart.safetensors #158 MB

# Softedge
# !{ARIA2} {HF_CTN_URL}/sargezt_xl_softedge.safetensors -d {CTN_MODEL_DIR} -o sargezt_xl_softedge.safetensors #2.5 GB

# Scribble
# !{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_scribble_anime.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_scribble_anime.safetensors #46.2 MB

# Sketch
# !{ARIA2} {HF_CTN_URL}/sai_xl_sketch_256lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_sketch_256lora.safetensors #774 MB
# !{ARIA2} {HF_CTN_URL}/sai_xl_sketch_128lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_sketch_128lora.safetensors #396 MB

# Segmentaion
# !{ARIA2} https://civitai.com/api/download/models/154866 -d {CTN_MODEL_DIR} -o Segmentaion.safetensors #84.3 MB

# Recolor
# !{ARIA2} {HF_CTN_URL}/sai_xl_recolor_256lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_recolor_256lora.safetensors #774 MB
# !{ARIA2} {HF_CTN_URL}/sai_xl_recolor_128lora.safetensors -d {CTN_MODEL_DIR} -o sai_xl_recolor_128lora.safetensors #396 MB

# ip-adapter
# !{ARIA2} {HF_CTN_URL}/ip-adapter_xl.pth -d {CTN_MODEL_DIR} -o ip-adapter_xl.pth #703 MB
!{ARIA2} https://civitai.com/api/download/models/155532 -d {CTN_MODEL_DIR} -o h94-ip-adapter-xl.pth #670 MB

# Blur
!{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_blur.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_blur.safetensors #46.2 MB
# !{ARIA2} {HF_CTN_URL}/kohya_controllllite_xl_blur_anime.safetensors -d {CTN_MODEL_DIR} -o kohya_controllllite_xl_blur_anime.safetensors #46.2 MB


# ip-adapter, revision のダウンロードファイル ---------------------------------------------------
CTN_DOWNLOADS = "/home/studio-lab-user/ui/extensions/sd-webui-controlnet/annotator/downloads"
!rm -fr {CTN_DOWNLOADS}/* #ControlNet ダウンロードファイル を全て削除（リセット）
# !{ARIA2} https://huggingface.co/lllyasviel/Annotators/resolve/main/clip_g.pth -d {CTN_DOWNLOADS}/clip_vision -o clip_g.pth #3.44 GB


# ADetailer Model の追加フォルダ ---------------------------------------------------
ADETAILER_MODEL_DIR = "/home/studio-lab-user/ui/models/adetailer"
!rm -fr {ADETAILER_MODEL_DIR}/* #ADetailer Model を全て削除（リセット）
!{ARIA2} https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt -d {ADETAILER_MODEL_DIR} -o hand_yolov8s.pt

# 確認
!echo -= CTN_MODEL_DIR を確認 =- && ls -l {CTN_MODEL_DIR}
!echo -= CTN_DOWNLOADS を確認 =- && ls -l {CTN_DOWNLOADS}
!echo -= ADETAILER_MODEL_DIR を確認 =- && ls -l {ADETAILER_MODEL_DIR}

#### ***画像生成 の結果をZipにする***

In [ ]:
# 画像生成 の結果をZipにする。100件以上のフォルダは、100件ごとにZipにする。

import os
import glob
import datetime
import zipfile
import shutil

# 今日日付を取得
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# 圧縮したい画像のパス
# 画像のパス
img_paths = [f"/home/studio-lab-user/ui/outputs/{folder}" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]

# ZIPファイルのパス
zip_paths = [f"/home/studio-lab-user/outputs_{folder}_{today}.zip" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]

# 移動対象のフォルダのパス
move_paths = [f"{img_path}/*" for img_path in img_paths]

# 不要なフォルダのパス
delete_paths = [f"/home/studio-lab-user/Delete/{folder}/{today}" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]


# ZIPファイルに圧縮する関数
def zip_files(img_path, zip_path):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        # サブフォルダとファイルの全てのパスを取得
        for root, dirs, files in os.walk(img_path):
            for file in files:
                # ZIPファイルに書き込むパスを相対パスにする
                rel_path = os.path.relpath(os.path.join(root, file), img_path)
                # ZIPファイルに書き込む
                zf.write(os.path.join(root, file), rel_path)


# img_pathのファイル数が50以上あれば、1, 2のような名前でフォルダを作って、100件ずつ移動させる関数
def split_files(img_path):
    # img_pathのファイル数を取得
    file_count = len(os.listdir(img_path))

    # ファイル数が50以上ならば
    if file_count > 50:
        # フォルダの番号を初期化
        folder_num = 1
        # ファイルのリストを取得
        file_list = os.listdir(img_path)
        # ファイルのリストを100件ずつに分割
        file_chunks = [file_list[i:i+50] for i in range(0, file_count, 50)]
        # 分割したファイルのリストごとに処理
        for file_chunk in file_chunks:
            # フォルダの名前を作成
            folder_name = str(folder_num)
            # フォルダのパスを作成
            folder_path = os.path.join(img_path, "zip_"+folder_name)
            # フォルダがなければ作成
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            # フォルダにファイルを移動
            for file in file_chunk:
                file_path = os.path.join(img_path, file)
                shutil.move(file_path, folder_path)
            # フォルダの番号を更新
            folder_num += 1


# 画像のパスとZIPファイルのパスのペアごとに圧縮と移動を行う関数
def zip_and_move(img_path, zip_path, move_path, delete_path):
    # 画像のパスに何もフォルダやファイルがなければスキップする
    # if len(os.listdir(img_path)) == 0:
    if not os.path.exists(img_path) or len(os.listdir(img_path)) == 0:
        return

    # img_pathのファイル数が100以上あれば、1, 2のような名前でフォルダを作って、100件ずつ移動させる関数を呼び出す
    split_files(img_path)

    # img_pathのサブフォルダのリストを取得
    subfolder_list = os.listdir(img_path)

    # サブフォルダのリストごとに処理
    for subfolder in subfolder_list:
        # サブフォルダのパスを作成
        subfolder_path = os.path.join(img_path, subfolder)
        # Zipファイルの名前を作成（末尾にサブフォルダの名前を付け足す）
        zip_name = zip_path[:-4] + "_" + subfolder + ".zip"
        # Zipファイルに圧縮する関数を呼び出す
        zip_files(subfolder_path, zip_name)

    # 移動先のディレクトリがなければ作成する
    if not os.path.exists(delete_path):
        os.makedirs(delete_path)

    # ワイルドカードにマッチするパスのリストを取得
    move_list = glob.glob(move_path)

    # 移動を実行
    for file in move_list:
        shutil.move(file, delete_path)

    print("実行 → ", img_path)
    !ls {img_path}

# 画像のパスとZIPファイルのパスのペアごとに圧縮と移動を行う
for img_path, zip_path, move_path, delete_path in zip(img_paths, zip_paths, move_paths, delete_paths):
    zip_and_move(img_path, zip_path, move_path, delete_path)

In [ ]:
# 対象のファイルを Zip にする。
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

img_path = "圧縮対象のファイル" # 例 : f"/home/studio-lab-user/outputs-{today}.zip"

zip_path = "圧縮ファイル名" # 例 : Delete/txt2img-images/20230816_130141/2023-08-16/2

zip_files("/home/studio-lab-user/"+img_path,zip_path)

#### ***容量の整理***

In [ ]:
"""
削除しないものはコメントアウトする。
コメントアウトの方法 → 行頭に『# 』をつける。
"""
# Stable Diffusion WebUI の 10MB 以上のファイルの場所とサイズを降順に列挙する。
%cd /home/studio-lab-user/ui
!find . -size +10M -print0 | du -ch --files0-from=- | sort -hr

# bak の中身を全て消す ---------------------------------------------------
!rm -fr ./bak/*

# 実行時に自動ダウンロードされるファイルを削除する ---------------------------------------------------
!rm -fr ./extensions/sd-webui-controlnet/annotator/downloads/* 
!rm -f ./models/BLIP/model_base_caption_capfilt_large.pth
!rm -f ./models/torch_deepdanbooru/model-resnet_custom_v3.pt

# 各モデルファイルを全て消す ---------------------------------------------------
# !rm -f ./ui/models/Stable-diffusion/* # Model
# !rm -f ./ui/models/Lora/* # LoRA
# !rm -f ./ui/models/ControlNet/* # Controlnet

# 対象のファイルを削除したい場合 ---------------------------------------------------
# !rm -f <削除したいファイルのパス>

# /dev/nvme2n1 で容量を確認
!df -h

In [ ]:
# Stable Diffusion WebUI 以外の容量の整理
!rm -rf /home/studio-lab-user/.ipynb_checkpoints
!rm -rf /home/studio-lab-user/.cache # キャッシュの削除
# !rm -rf /home/studio-lab-user/.conda #リセット（最初からやり直すなら実行）

# /dev/nvme2n1 で容量を確認
!df -h

#### ***環境のリセット***

In [ ]:
# 環境をリセット（削除）
conda activate sd_env
conda deactivate
conda remove -y -n sd_env --all

#### ***AUTOMATIC1111アップデート（アップデートが必要な時のみ実行）***

In [ ]:
# Stable Diffusion WebUI を最新版にアップデートする。
%cd /notebooks/ui
!git checkout master
!git pull